# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""../../../../da…","""10718750946070…",1.2927e19,"""94697505469765…","""74380109928052…","""16923848102310…",false
"""../../../../da…","""65280004865854…",1.4856e19,"""41809198419622…","""74380109928052…","""39168150070413…",true
"""../../../../da…","""13207102415811…",5.6458e17,"""87518970726774…","""12911659601947…","""16923848102310…",false
"""../../../../da…","""65280004865854…",4.0723e18,"""65188162850855…","""11146528333082…","""16923848102310…",false
"""../../../../da…","""10718750946070…",1.4677e19,"""17448516922160…","""11146528333082…","""16719304484719…",true
"""../../../../da…","""13207102415811…",1.6330e19,"""55014544439398…","""14033878408871…",null,false
"""../../../../da…","""13207102415811…",8.7675e18,"""85868064371213…","""11146528333082…","""16719304484719…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../da…","""10741749796594…",1.2927e19,"""66116287026678…","""FirstMortgage3…","""13578844711837…","""Rejected"""
"""../../../../da…","""47552813521801…",1.4856e19,"""13532252447934…","""FirstMortgage3…","""16178372471158…","""Accepted"""
"""../../../../da…","""85502469574282…",5.6458e17,"""80406127316873…","""MoneyMarketSav…","""13578844711837…","""Rejected"""
"""../../../../da…","""47552813521801…",4.0723e18,"""34649052124215…","""BasicChecking""","""13578844711837…","""Rejected"""
"""../../../../da…","""10741749796594…",1.4677e19,"""11381590245212…","""BasicChecking""","""27933990805361…","""Accepted"""
"""../../../../da…","""85502469574282…",1.6330e19,"""99423387406972…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""85502469574282…",8.7675e18,"""95967152498326…","""BasicChecking""","""27933990805361…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../da…","""18244738254682…",1.2927e19,"""25395181669605…","""FirstMortgage3…","""38835902423253…","""Rejected"""
"""../../../../da…","""41388947394791…",1.4856e19,"""12089617175462…","""FirstMortgage3…","""83121342572697…","""Accepted"""
"""../../../../da…","""15723207924199…",5.6458e17,"""67793278007536…","""MoneyMarketSav…","""38835902423253…","""Rejected"""
"""../../../../da…","""41388947394791…",4.0723e18,"""47646165980697…","""BasicChecking""","""38835902423253…","""Rejected"""
"""../../../../da…","""18244738254682…",1.4677e19,"""10670960375960…","""BasicChecking""","""65645554489736…","""Accepted"""
"""../../../../da…","""15723207924199…",1.6330e19,"""17682316635075…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""15723207924199…",8.7675e18,"""63613092107560…","""BasicChecking""","""65645554489736…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_MaritalStatus=PREDICTOR_1
Customer_CLV=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""11678886745887…",1.2927e19,"""12764835835456…","""FirstMortgage3…","""43417772782832…","""Rejected"""
"""49784409043961…",1.4856e19,"""26392466716667…","""FirstMortgage3…","""86891274243646…","""Accepted"""
"""90016740327842…",5.6458e17,"""29274778688186…","""MoneyMarketSav…","""43417772782832…","""Rejected"""
"""49784409043961…",4.0723e18,"""46041789384268…","""BasicChecking""","""43417772782832…","""Rejected"""
"""11678886745887…",1.4677e19,"""96675387296566…","""BasicChecking""","""64155491140682…","""Accepted"""
"""90016740327842…",1.6330e19,"""48734140923004…","""UPlusFinPerson…",null,"""Rejected"""
"""90016740327842…",8.7675e18,"""16504441058506…","""BasicChecking""","""64155491140682…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'